In [1]:
import pickle
import os
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
from sklearn.metrics import roc_auc_score

In [3]:
#on VM
os.chdir('/mnt/mypartition/Desktop2/pubmed_nlp_research/DeepLearningMovies_datasets/')
import KaggleWord2VecUtility

In [2]:
#on local:
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [3]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [4]:
#df = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/article_replication/df.p', 'rb'))
df = pickle.load(open('/mnt/mypartition/Desktop2/dfJournals_trans_categories.p', 'rb'))

In [5]:
#for the journals VM dataframe, change the label of the 'sentiment' column to 'label'
df.columns = ['label', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']

In [6]:
print df.shape
print df.columns.values

(97049, 5)
['label' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms']


In [20]:
df['boolean_label'] = 1
for row in df.iterrows():
    if row[1]['label'] != 'T1/T2':
        df.set_value(row[0], 'boolean_label', 0)

In [8]:
df

,label,title,abstract,qualifier_terms,descriptor_terms,boolean_label
0,T0,Quantification of the relative contributions o...,Amyotrophic lateral sclerosis (ALS) and fronto...,,,1
1,T0,Ubiquitin Specific Protease 36 (USP36) control...,Ubiquitination of the TrkA neurotrophin recept...,,,1
2,T0,Functional Diversity of Cytotoxic tRNase/immun...,Contact-dependent growth inhibition (CDI) is a...,,,1
3,T0,Identification of a distinct substrate binding...,The type III secretion system (T3SS) effector ...,,,1
4,T0,Role of chondroitin sulfate (CS) modification ...,Protein tyrosine phosphatase receptor type Z (...,,,1
5,T0,Chemically programmed bispecific antibodies in...,Chemically programmed bispecific antibodies (b...,,,1
6,T0,Non-mutagenic Suppression of Enterocyte Ferrop...,Iron transfer across the basolateral membrane ...,,,1
7,T0,Structural plasticity of cholesteryl ester tra...,Cholesteryl ester transfer protein (CETP) medi...,,,1
8,T0,Identification of a Membrane-Bound Prepore Spe...,Pore-forming toxins (PFT) are cytolytic protei...,,,1
9,T0,EspR-dependent ESAT-6 secretion of Mycobacteri...,Attenuation of M. bovis BCG strain is related ...,,,1


## Apply word2vec inversion

In [13]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

import pandas as pd # for quick summing within doc

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [18]:
a = datetime.datetime.now().replace(microsecond=0)

#5-fold stratified cross validation
from sklearn.cross_validation import StratifiedKFold

#because no validation set, 4/5 of values go to train and 1/5 to test
#is this too high?***
#even though we shuffle, not as randomly distributed as the former method was
y = df['boolean_label'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

i = 1
avg = []
for train_index, test_index in skf:

    print("ROUND", i)
    i = i + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]

    # ****** Split the training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    #here change to include all journal name labels of positive and negative
    inxs_pos = np.where(train1['boolean_label'] == 1)[0].tolist()
    inxs_neg = np.where(train1['boolean_label'] == 0)[0].tolist()

    print "Parsing sentences from training set"
    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        title = train1['title'].iloc[inx]
        #descriptors = train1['descriptor_terms'].iloc[inx]
        #qualifiers = train1['qualifier_terms'].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        #sentences_pos += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
        #sentences_pos += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)


    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        title = train1['title'].iloc[inx]
        #descriptors = train1['descriptor_terms'].iloc[inx]
        #qualifiers = train1['qualifier_terms'].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        #sentences_neg += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
        #sentences_neg += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)

    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences

    for j in range(len(train1)):
        review = train1["abstract"].iloc[j]
        title = train1['title'].iloc[j]
        #descriptors = train1['descriptor_terms'].iloc[j]
        #qualifiers = train1['qualifier_terms'].iloc[j]

        sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        #sentences += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
        #sentences += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)

    print "Building and training w2v models"
    ## create a w2v learner 
    basemodel = Word2Vec(
        workers=multiprocessing.cpu_count(), # use your cores
        iter=3, # iter = sweeps of SGD through the data; more is better
        hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
    basemodel.build_vocab(sentences) 

    #train models
    models = [deepcopy(basemodel) for y in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )

    print "Parsing test sentences"
    # read in the test set as a list of a list of words

    docs = []
    for k in range(len(test1)):
        sents = []
        review = test1["abstract"].iloc[k]
        title = test1['title'].iloc[k]
        #descriptors = test1['descriptor_terms'].iloc[k]
        #qualifiers = test1['qualifier_terms'].iloc[k]

        sents += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sents += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        #sents += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
        #sents += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)
        docs.append(sents)

    print "scoring test set"
    # get the probs (note we give docprob our test set plus the models)
    probs = docprob(docs,models).astype(object)

    predictions = np.ones((probs.shape[0]))

    predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
    print predictions
    print np.size(predictions)

    score = roc_auc_score(test1["boolean_label"], predictions)
    #score = np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)
    print score
    #append to average
    avg.append(score)


print("average of 5 rotations:", sum(avg)/float(len(avg)))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 1.  1.  1. ...,  0.  0.  0.]
19411
0.942054288636
('ROUND', 2)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 1.  1.  1. ...,  0.  0.  0.]
19410
0.940347429807
('ROUND', 3)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 1.  1.  1. ...,  0.  0.  0.]
19410
0.940635006981
('ROUND', 4)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 1.  1.  1. ...,  0.  0.  0.]
19409
0.941390166797
('ROUND', 5)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 1.  1.  1. ...,  0.  0.  0.]
19409
0.940910839991
('average of 5 rotations:', 0.94106754644240653)
0:46:17


## Performance with abstract, title, and mesh terms on PAPER'S DATASET (much worse than BOWs/RF because so few records)
T0 class- 0.68 (0.63, 0.73)

T1/T2 class- 0.49 (0.45, 0.52)

T3/T4 class- 0.51 (0.48, 0.55)

## Performance with abstract, title, and mesh terms on PUBMED JOURNALS (performs how compared to BOWs/RF?)
T0 class- 0.94

T1/T2 class- 0.82

T3/T4 class- 0.88

### Import KaggleWord2VecUtility since didn't work from file

In [2]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences